Создадим функции преобразования для данных смс. Номера телефонов 11 знаков (пример: 09061701461), есть e-mail'ы и ссылки. Их меняем на токены ('§p' для номера телефона, '§m' для адреса почты, '§w' для ссылки, '§d' для дат)

In [1]:
import re
def get_text(text):
    #return re.sub(r'[^a-z §p§m§w§n§d]', '', re.sub(r'(\d?\d.?st)|(\d?\d.?nd)|(\d?\d.?rd)|(\d?\d.?th)', ' §d ',  text.lower()))
    #return re.sub(r'(\d?\d.?st)|(\d?\d.?nd)|(\d?\d.?rd)|(\d?\d.?th)', ' §d ', re.sub(r'(0\d{10})', ' §p ', text.lower()))
    #return re.sub(r'[^a-z §p§m§w§n§d]', '', re.sub(r'(\d?\d.?st)|(\d?\d.?nd)|(\d?\d.?rd)|(\d?\d.?th)', ' §d ', re.sub(r'(0\d{10})', ' §p ', text.lower())))
    #return re.sub(r'[^a-z §p§m§w§n§d]', '', re.sub(r'(\d?\d.?st)|(\d?\d.?nd)|(\d?\d.?rd)|(\d?\d.?th)', ' §d ', text.lower()))

    #return re.sub(r'[^a-z §p§m§w§n§d]', '', re.sub(r'(\d?\d.?st)|(\d?\d.?nd)|(\d?\d.?rd)|(\d?\d.?th)', ' §d ', re.sub(r'(\S+\.(\w{2,3})\S+)', ' §w ', re.sub(r'(www.+\.(\w{2,3})\S+)|(wap.+\.(\w{2,3})\S+)', ' §w ', re.sub(r'(htt.+\.(\w{2,3})\S+)', ' §w ', re.sub(r'(\w+)@(\w+)\.(\w{2,3})', ' §m ', text.lower()))))))
    #return re.sub(r'[^a-z §p§m§w§n§d]', '', re.sub(r'(\S+\.(\w{2,3})\S+)', ' §w ', re.sub(r'(www.+\.(\w{2,3})\S+)|(wap.+\.(\w{2,3})\S+)', ' §w ', re.sub(r'(htt.+\.(\w{2,3})\S+)', ' §w ', re.sub(r'(0\d{10})', ' §p ',re.sub(r'(\w+)@(\w+)\.(\w{2,3})', ' §m ', text.lower()))))))
    return re.sub(r'[^a-z §p§m§w§n§d]', '', re.sub(r'\d', ' §n ',re.sub(r'(\d?\d.?st)|(\d?\d.?nd)|(\d?\d.?rd)|(\d?\d.?th)', ' §d ', re.sub(r'(\S+\.(\w{2,3})\S+)', ' §w ', re.sub(r'(www.+\.(\w{2,3})\S+)|(wap.+\.(\w{2,3})\S+)', ' §w ', re.sub(r'(htt.+\.(\w{2,3})\S+)', ' §w ', re.sub(r'(0\d{10})', ' §p ',re.sub(r'(\w+)@(\w+)\.(\w{2,3})', ' §m ', text.lower()))))))))


Разные комбинации регулярных выражений (в том же порядке, что и в коде) дают разные результаты на 5-fold validation с roc auc
1. 0.5477 (0.9372)
2. 0.5003 (0.9298)
3. 0.5551 (0.9357)
4. 0.5477 (0.9372)
5. 0.5513 (0.9345)
6. 0.5581 (0.9360)
7. 0.4939 (0.9470)

Значения в начале - для анализа отдельных слов, в скобках - для словосочетаний

Видим, что 7 вариант оптимален, его и будем использовать.
Это фильтрация почт, телефонов, сайтов, дат и цифр

Вот отдельно выписанные регулярные выражения, которые там используются:

In [ ]:
re.sub(r'(\w+)@(\w+)\.(\w{2,3})', ' §m ', string[4:])

Выполнится первым и удалит все адреса почты.

In [ ]:
 re.sub(r'(0\d{10})', ' §p ', ...)

Выполнится вторым (на результате первого) и удалит все номера телефонов.

In [ ]:
re.sub(r'(\S+\.(\w{2,3})\S+)', ' §w ', re.sub(r'(www.+\.(\w{2,3})\S+)|(wap.+\.(\w{2,3})\S+)', ' §w ', re.sub(r'(htt.+\.(\w{2,3})\S+)', ' §w ', ...)))

Выполнится далее (на результате второго) и удалит все адреса сайтов (все варианты записи, которые я нашёл).

In [ ]:
re.sub(r'\d', ' §n ', ...)

Выполняется далее и меняет оставшиеся в тексте цифры на спецсимвол

In [ ]:
re.sub(r'(\d?\d.?st)|(\d?\d.?nd)|(\d?\d.?rd)|(\d?\d.?th)', ' §d ', ...)

Меняет все даты на спецсимвол

Парсим коллекцию.

In [2]:
f = open('SMSSpamCollection/SMSSpamCollection')
collection_txt = list()
collection_spam = list()
for line in f:
    if line[:3] == 'ham':
        collection_spam.append('0')
        text = get_text(line[4:])
    else:
        collection_spam.append('1')
        text = get_text(get_text(line[5:]))
    collection_txt.append(text)
    
    

Функция инициализации Байесовского классификатора. 
Создаёт пустые поля **prior** и __condprob__ . 
+ __С__ - *словарь*, ключи которого есть темы (*spam*/*ham*), а значения - слова, встреченные в соответсвующих темах;


Функция обучения Байесовского классификатора.  
Аргументы: 
+ **x_** - _список_, список текстов;
+ **y_** _список_, список категорий 

Результат (сохраняется внутри модели, функция не возвращает ничего):  
+ **prior** - вероятность появления темы
+ **condprob** - вероятность каждого слова в каждой теме


Функция работы Байесовского классификатора.  
Аргументы: 
+ __d__ - текст документа

Результат:  
+ *текст* - категория, в данной задаче 0 - *ham*, а 1 - *spam*

Классификатор, приведённый здесь, может работать с более, чем двумя темами
Если попытаться запустить классификатор без обучения (с пустыми **prior** и **condprob**) случится RuntimeError

In [3]:
import progressbar
import numpy as np
from collections import Counter
from ctypes import *
import math
import operator

class naive_Bayes:
    def __init__(self, c = None):
        self.c = c
        self.prior = None
        self.condprob = None
    
    def __get_w(self, i, j):
        a = self.condprob[i][j]
        if a is None:
            return 1
        else:
            return a
    
    def fit(self, x_, y_):
        if len(x_) == len(y_):
            n = len(x_)
        else:
            raise RuntimeError('Invalid arguments')
        v = set()  
        if self.c == None:
            self.c = {'0': set(), '1': set()}
            for i in range(n):
                words = x_[i].split(' ')
                if y_[i] == '0':
#                    self.c['0'].update(set(x_[i].split(' ')))
                    self.c['0'].update(set(tuple(zip([''] + words, words))))
                else:
#                    self.c['1'].update(set(y_[i].split(' ')))
                    self.c['1'].update(set(tuple(zip([''] + words, words))))
        
        [v.update(set(x)) for x in self.c.values()] # словарь
        self.prior = dict()
        self.condprob = dict()
        widgets = [progressbar.Percentage(), progressbar.Bar()]
        bar = progressbar.ProgressBar(widgets=widgets, max_value=len(self.c.keys())).start()
        i_bar = 0
        themes_count = Counter(y_) # считаем частоты появления тем
        for i in self.c.keys():
            self.prior.update(dict.fromkeys([i], themes_count[i]/n))
        # ----------------------------------------
            text = ''
            text += ' '.join(np.array(x_)[np.isin(np.array(y_), i)]) # собираем все тексты 
                                                                                 # из одной категории в один общий
            text = text.split(' ')
            text = list(tuple(zip([''] + text, text)))
            #text = list(tuple(zip([''] + [''] + text, [''] + text, text)))
        # в text собираются тела/заголовки документов, отмеченных темой i
        # ----------------------------------------
            if len(text) == 0:
                continue
            word_count = Counter(text) # считаем частоты слов в общем тексте
            all_ = len(text) + len(v)
            self.condprob[i] = dict()
            for x in v:
                self.condprob[i].update(dict.fromkeys([x], ((word_count[x] + 1) / all_))) # вероятность каждого слова в каждой теме
            bar.update(i_bar)
            i_bar += 1
        bar.finish()
    
    def use(self, d):
        if self.prior == None:
            raise RuntimeError('Model is not fitted!') 
        body = get_text(d).split()
        v = set()
        [v.update(set(x)) for x in self.c.values()]
        w = list()
        got_text = list(tuple(zip([''] + body, body)))
        #got_text = list(tuple(zip([''] + [''] + text, [''] + text, text)))
        for i in got_text:
            if i in v:
                w.append(i)
        score = dict()
        for i in self.c.keys():
            if self.prior[i] > 0:
                score.update(dict.fromkeys([i], (math.log(self.prior[i]))))
                for j in w:
                    score[i] += math.log(self.__get_w(i,j)) #self.condprob[i][j]
        return max(score.items(), key=operator.itemgetter(1))[0]
    
    def predict(self, arr):
        return [self.use(i) for i in arr]
    
    def get_params(self, deep=True): # Сделано только что бы cross_val_score работал
        return dict()
    
    


In [4]:
clf = naive_Bayes()

Посчитаем _accuracy_, чтобы убедится, что модель работает, хотя эта оценка некорректна для данной задачи

In [5]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, collection_txt,collection_spam,scoring = 'accuracy',cv=5)

100%|#########################################################################|
100%|#########################################################################|
100%|#########################################################################|
100%|#########################################################################|
100%|#########################################################################|


In [6]:
print('E: ' + str(scores.mean()))
print('+/-: '+ str(scores.std()))

E: 0.981521604366763
+/-: 0.00257277684583844


Используем ___roc auc - меру___ так как классы не взвешены

In [7]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

lb1 = LabelBinarizer()
lb2 = LabelBinarizer()

def scorer(estimator, X, y):
    return roc_auc_score(lb1.fit_transform(y), lb2.fit_transform(estimator.predict(X)))


In [8]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, collection_txt,collection_spam,scoring = scorer,cv=5)

100%|#########################################################################|
100%|#########################################################################|
100%|#########################################################################|
100%|#########################################################################|
100%|#########################################################################|


In [9]:
print('E: ' + str(scores.mean()))
print('+/-: '+ str(scores.std()))

E: 0.947131738130043
+/-: 0.007083365110202644


Использование словосочетаний вместо отдельных слов позволило увеличить точность с 55% до 94%

Для сравнения запустим Байесовский классификатор из sklearn. Рассмотрим также ___roc auc - меру___ 

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
collection_txt2 = vect.fit_transform(collection_txt)

from sklearn.naive_bayes import MultinomialNB
clfsk = MultinomialNB()

scores = cross_val_score(clfsk, collection_txt2,collection_spam,scoring = 'roc_auc',cv=5)

In [11]:
print('E: ' + str(scores.mean()))
print('+/-: '+ str(scores.std()))

E: 0.9781327409873836
+/-: 0.010434947385387203


И для сравнения _accuracy_

In [12]:
scores = cross_val_score(clfsk, collection_txt2,collection_spam,scoring = 'accuracy',cv=5)

In [13]:
print('E: ' + str(scores.mean()))
print('+/-: '+ str(scores.std()))

E: 0.9745220491238907
+/-: 0.0030539424212447422


__ВЫВОД:__ написанный собственноручно метод уступает на 3% аналогу из библиотеки по roc-auc метрике, а по accuracy даже немного превосходит